In [3]:
from scipy.spatial import ConvexHull
import numpy as np
from math import sqrt
from itertools import combinations

In [4]:
allPoints = []

n=15 # number of points
N=10 # +/- range of points 

for i in range(n):
    points = (float(np.random.randint(-N,N)),float(np.random.randint(-N,N)))
    allPoints.append(points)
# creates list of random points

allPoints = list(set(allPoints)) #takes out possible duplicates

print("allPoints:",allPoints,"\n")

allPoints: [(-4.0, 5.0), (0.0, -5.0), (-1.0, 1.0), (5.0, 1.0), (6.0, 8.0), (6.0, 6.0), (7.0, -9.0), (1.0, 4.0), (-9.0, 1.0), (-1.0, -4.0), (-2.0, -1.0), (0.0, -1.0), (-5.0, 2.0), (-4.0, -7.0), (-10.0, 0.0)] 



In [9]:
class TravelPoly:
    def __init__(self):
        self.OP = [allPoints[i] for i in ConvexHull(allPoints).vertices]
        print("OP:",self.OP,"\n") # Prints points of convex hull (OP)
        
        self.IP = [x for x in allPoints if x not in self.OP]
        print("IP:",self.IP,"\n"*2) # Prints all inner points (not in OP)

        self.outerDistsDict = {} 
        # Is a dict because distance is from OP to IP
        # Whereas the direction of IP to other IP is not determined
        
        self.innerDistsList = [] 
        # List of tuples because the direction between points isn't known
        # Can be changed depending on starting conditions
        
        # TODO self.touchedIP = []

        self.getOuterDistsDict() # Gets all distances from OP to IP
        self.getInnerDistsList() # Gets all distances from IP to other IP

    def algorithm(self):
        '''
        TODO WHILE there are IP (inner points) not connected by two edges
        '''
        for i in range(10): # TODO replace FOR with WHILE loop 
            self.getMinOR() # Gets smallest outer radius (from OP to IP)
            self.listIR = self.listOR # Adds that OR to the list of IR
            
            self.getListIR(self.listOR) 
            # Finds next closest IP from most recently connected point(s) 
            # Skips IP that may be within IR, thought upThroughIR fixes this
            # TODO May be able to include these in one step
            self.listIR.sort(key=lambda tup: tup[1][1])
            print("listIR:",self.listIR)
            
            # Probably not necessary:
#             self.getListIR(self.listIR) 
#             self.listIR.sort(key=lambda tup: tup[1][1])
#             print("listIR:",self.listIR)

            iterList = self.listIR.copy()
            self.listIR = []
            self.upThroughIR(iterList)
            # Checks for any points between smallest and second smallest IR
            # Since the IR is always the closest point
            # TODO May be easier to include all points smaller than current IR
                
    @staticmethod
    def distance(point1, point2): # distance formula
        return sqrt((point1[0]-point2[0])**2+(point1[1]-point2[1])**2)    

    def getOuterDistsDict(self): # Gets all distances from OP to IP
        print("outerDistsDict:")
        for a in self.OP:
            temp = []
            for b in self.IP:
                temp.append((b,self.distance(a,b)))
            
            temp.sort(key=lambda tup: tup[1])
            print(a,":",temp,"\n")
            self.outerDistsDict.update({a:temp})
            
    def getInnerDistsList(self): # Gets all distances from IP to other IP
        print("\n""innerDistsList:")
        for a,b in combinations(self.IP,2):
            self.innerDistsList.append(((a,b),self.distance(a,b)))
        
        self.innerDistsList.sort(key=lambda tup: tup[1])
        print(self.innerDistsList)

    def getMinOR(self): # Gets smallest outer radius (from OP to IP)
        self.listOR = [] 
        min_value = float("inf")
        for k,v in self.outerDistsDict.items():
            if v[0][1] == min_value:
                self.checkForMultiOR(k,v,min_value,self.listOR)
            elif v[0][1] < min_value:
                min_value = v[0][1]
                self.listOR = []
                self.checkForMultiOR(k,v,min_value,self.listOR)
        
        print("\n"*2,"listOR:",self.listOR)
        for key,val in self.listOR: # Removes minOR_list values from outerDistsDict dictionary
            self.outerDistsDict[key].remove(val)

    def checkForMultiOR(self,key,val,min_value,radiusList): # Checks if an OP connects to multiple IP
        radiusList.append((key,(val[0][0],val[0][1])))
        for i in range(len(val)-1):
            if val[i+1][1] == min_value:
                radiusList.append((key,(val[i+1][0],val[i+1][1])))
            else:
                break
        
    def checkForIR(self,radiusList): # Finds next closest IP from most recently connected point(s)      
        temp = []
        for (i,j),k in self.innerDistsList:
            for a,(b,c) in radiusList:
                if k <= c:
                    if b == j:
                        if not temp:
                            temp.append((b,(i,k)))
                        elif k == temp[0][1][1]:
                            temp.append((b,(i,k)))
                        elif k < temp[0][1][1]:
                            temp = []
                            temp.append((b,(i,k)))
                    elif b == i:
                        if not temp:
                            temp.append((b,(j,k)))
                        elif k == temp[0][1][1]:
                            temp.append((b,(j,k)))
                        elif k < temp[0][1][1]:
                            temp = []
                            temp.append((b,(j,k)))
                else:
                    return temp                  

    def getListIR(self,radiusList): # Finds next closest IP from most recently connected point(s) 
        temp = self.checkForIR(radiusList)
        self.listIR += temp  
        print("temp:",temp,"\n")
        for a,(b,c) in temp: # Removes temp values from innerDistsList dictionary
            try:
                self.innerDistsList.remove(((a,b),c))
            except:
                try:
                    self.innerDistsList.remove(((b,a),c))
                except:
                    pass #TODO
                    
        if temp:
            self.getListIR(temp) # recursive

    def upThroughIR(self,iterList): # Checks for any points between smallest and second smallest IR
        del iterList[0]
        if iterList:
            self.getListIR(iterList)
            iterList = self.listIR + iterList
            self.listIR = []
            print("iterList:",iterList)
            self.upThroughIR(iterList) # recursive  

    def updateOP(self,listName):
        #for a,(b,c) in listName:
        pass
    
    def getTouchedPoints(self):
        #for val in self.listIR:
        pass

    def updateEdges(self):
        #for val in self.listIR:
        pass

    def breakEdges(self):
        pass

In [10]:
TravelPoly().algorithm()

OP: [(7.0, -9.0), (6.0, 8.0), (-4.0, 5.0), (-9.0, 1.0), (-10.0, 0.0), (-4.0, -7.0)] 

IP: [(0.0, -5.0), (-1.0, 1.0), (5.0, 1.0), (6.0, 6.0), (1.0, 4.0), (-1.0, -4.0), (-2.0, -1.0), (0.0, -1.0), (-5.0, 2.0)] 


outerDistsDict:
(7.0, -9.0) : [((0.0, -5.0), 8.06225774829855), ((-1.0, -4.0), 9.433981132056603), ((5.0, 1.0), 10.198039027185569), ((0.0, -1.0), 10.63014581273465), ((-2.0, -1.0), 12.041594578792296), ((-1.0, 1.0), 12.806248474865697), ((1.0, 4.0), 14.317821063276353), ((6.0, 6.0), 15.033296378372908), ((-5.0, 2.0), 16.278820596099706)] 

(6.0, 8.0) : [((6.0, 6.0), 2.0), ((1.0, 4.0), 6.4031242374328485), ((5.0, 1.0), 7.0710678118654755), ((-1.0, 1.0), 9.899494936611665), ((0.0, -1.0), 10.816653826391969), ((-2.0, -1.0), 12.041594578792296), ((-5.0, 2.0), 12.529964086141668), ((-1.0, -4.0), 13.892443989449804), ((0.0, -5.0), 14.317821063276353)] 

(-4.0, 5.0) : [((-5.0, 2.0), 3.1622776601683795), ((-1.0, 1.0), 5.0), ((1.0, 4.0), 5.0990195135927845), ((-2.0, -1.0), 6.324555320336